<a href="https://colab.research.google.com/github/WalterTheDataAnalyst/Analysis-for-soccer-players/blob/main/Convolutional_Neural_Network_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import TensorFlow into the Program

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Best for:
Full flexibility and control

Custom training logic (e.g., custom loss functions, metrics, etc.)

Research and advanced experimentation

### Load Image

In [ ]:
mnist =tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


#Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the tf.keras model using the Keras model subclassing API:

In [ ]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation ='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x= self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()




Choose an optimizer and loss function for training:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
      # training=True is only needed if there are layers with different
      # behavior during training versus inference (e.g. Dropout).
      predictions = model(images, training=True)
      loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 8

for epoch in range(EPOCHS):
  #Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
      f'Epoch {epoch + 1},'
      f'Loss: {train_loss.result(): 0.2f},'
      f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
      f'Test Loss: {test_loss.result():0.2f}, '
      f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'

 )

Epoch 1,Loss:  0.01,Accuracy: 99.77, Test Loss: 0.06, Test Accuracy: 98.39
Epoch 2,Loss:  0.01,Accuracy: 99.82, Test Loss: 0.06, Test Accuracy: 98.52
Epoch 3,Loss:  0.00,Accuracy: 99.83, Test Loss: 0.07, Test Accuracy: 98.54
Epoch 4,Loss:  0.00,Accuracy: 99.86, Test Loss: 0.08, Test Accuracy: 98.44
Epoch 5,Loss:  0.00,Accuracy: 99.89, Test Loss: 0.08, Test Accuracy: 98.40
Epoch 6,Loss:  0.00,Accuracy: 99.89, Test Loss: 0.07, Test Accuracy: 98.64
Epoch 7,Loss:  0.00,Accuracy: 99.91, Test Loss: 0.08, Test Accuracy: 98.39
Epoch 8,Loss:  0.00,Accuracy: 99.91, Test Loss: 0.09, Test Accuracy: 98.53
